In [ ]:
from epx import Job, ModelConfig, SynthPop

import time

import plot_output as plot

# Running the Model

In [ ]:
flu_vacc_config = ModelConfig(
        synth_pop=SynthPop("US_2010.v5", ["Lassen_County_CA"]),
        start_date="2022-01-01",
        end_date="2031-12-30",
        seed=74708
    )

flu_vacc_job = Job(
    program="model/flu_vacc.fred",
    config=[flu_vacc_config],
    key="flu-vacc",
    fred_version="12.0.0",
    results_dir="/home/epx/cl-results"
)

In [ ]:
flu_vacc_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(flu_vacc_job.status) != 'DONE':
    if str(flu_vacc_job.status) == 'ERROR':
        logs = flu_vacc_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(flu_vacc_job.status)

# Exploring the Model Output

## Seasonal Epi-Curves

In [ ]:
epi_curves = plot.get_epi_curves(job)
epi_curves.show()

## Yearly Health/Immunity Status Animation 

In [ ]:
animation = plot.get_household_animation(job)
animation.show()

# Deleting the Job

In [ ]:
job.delete(interactive=False)